С помощью скрипта можно относительно быстро получить список отзывов к определенному объекту на Яндекс.Карты.

Для начала работы нам необходимо подгрузить модули для работы с сохраненной страницей в формате html.


In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

Далее нам нужно загрузить наш файл с сохраненной страницей в формате html, изменив путь в строке. Для этого в Colab найдите слева иконку папки и найдите кнопку загрузки файла (иконка под словом "Файлы"). Загрузив страницу, нажмите по загруженному файлу правой кнопкой мыши и выберите "Скопировать путь". В строке с кодом внизу замените запись на ту, что скопировали, предварительно удалив содержимое внутри скобок в кавычках.

In [ ]:
with open('/content/Зимний театр — Яндекс Карты.html') as f:
    content = f.read()

Включаем в работу модуль для обработки html-страниц BeautifulSoup.

In [ ]:
soup = BeautifulSoup(content, features='html.parser')

Функция для извлечения комментариев для Яндекс.Карт

In [ ]:
def get_reviews_from_YandexMaps(bs4_soup = soup):

    reviews = []
    authors = []
    authors_status = []
    authors_ratings = []
    publication_date = []

    for result in soup.findAll('div',\
                               class_='business-reviews-card-view__review'):
        try:
            reviews.append(result.find('span',\
                                       class_='business-review-view__body-text').\
                           get_text())
        except:
            reviews.append(np.NaN)

        try:
            authors.append(result.find('div',\
                                       class_='business-review-view__author').\
                           find('span', attrs={'itemprop': 'name'}).get_text())
        except:
            authors.append(np.NaN)
            
        try:
            authors_status.append(result.find('div',\
                                              class_='business-review-view__author').\
                                  find('div', class_='business-review-view__author-profession').\
                                  get_text())
        except:
            authors_status.append(np.NaN)
        
        try:
            authors_ratings.append(result.find('div', class_='business-review-view__rating').\
                                       find('span', attrs={'itemprop': 'reviewRating'}).\
                                       find('meta', attrs={'itemprop': 'ratingValue'})['content'])
        except:
            authors_ratings.append(np.NaN)
        
        try:
            publication_date.append(result.find('span', class_='business-review-view__date').\
                                    find('meta')['content'])
        except:
            publication_date.append(0)
    
    authors_ratings_int = [float(n) for n in authors_ratings]

    data = pd.DataFrame.from_dict({'author': authors,\
                               'status': authors_status,\
                               'rating': authors_ratings_int,\
                               'publication_date': publication_date,\
                               'review': reviews})
    data['publication_date'] = pd.to_datetime(data['publication_date']).dt.date

    return data


Переменная с обработанным функцией результатом парсинга страницы.

In [ ]:
data = get_reviews_from_YandexMaps(bs4_soup=soup)

In [ ]:
#data.head(5) # уберите "#" в начале строки, чтобы вывести первые пять строк полученной таблицы.

Скрипт сохраняет данные в формате .xlsx в раздел "Файлы", откуда затем файлы можно скачать к себе на рабочий стол.

In [ ]:
data.to_excel('result.xlsx')